# This dataset contains information on historic trades for several cryptoassets, such as Bitcoin and Ethereum. Your challenge is to predict their future returns.

As historic cryptocurrency prices are not confidential this will be a forecasting competition using the time series API. Furthermore the public leaderboard targets are publicly available and are provided as part of the competition dataset. Expect to see many people submitting perfect submissions for fun. Accordingly, THE PUBLIC LEADERBOARD FOR THIS COMPETITION IS NOT MEANINGFUL and is only provided as a convenience for anyone who wants to test their code. The final private leaderboard will be determined using real market data gathered after the submission period closes.

train.csv - The training set

- timestamp - A timestamp for the minute covered by the row.
- Asset_ID - An ID code for the cryptoasset.
- Count - The number of trades that took place this minute.
- Open - The USD price at the beginning of the minute.
- High - The highest USD price during the minute.
- Low - The lowest USD price during the minute.
- Close - The USD price at the end of the minute.
- Volume - The number of cryptoasset u units traded during the minute.
- VWAP - The volume-weighted average price for the minute.
- Target - 15 minute residualized returns. See the 'Prediction and Evaluation section of this notebook for details of how the target is calculated.
- Weight - Weight, defined by the competition hosts here
- Asset_Name - Human readable Asset name.

example_test.csv - An example of the data that will be delivered by the time series API.

example_sample_submission.csv - An example of the data that will be delivered by the time series API. The data is just copied from train.csv.

asset_details.csv - Provides the real name and of the cryptoasset for each Asset_ID and the weight each cryptoasset receives in the metric.

supplemental_train.csv - After the submission period is over this file's data will be replaced with cryptoasset prices from the submission period. In the Evaluation phase, the train, train supplement, and test set will be contiguous in time, apart from any missing data. The current copy, which is just filled approximately the right amount of data from train.csv is provided as a placeholder.

    📌 There are 14 coins in the dataset

    📌 There are 4 years in the [full] dataset


# Code

In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import time

In [2]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print(use_cuda)

False


# Load data

In [4]:
pd.set_option('display.max_rows', None)
train_csv = pd.read_csv("train.csv")
print(len(train_csv))
test_csv = pd.read_csv("example_test.csv")
print(len(test_csv))
samples_submission_csv = pd.read_csv("example_sample_submission.csv")

24236806
56


# Memory review

In [6]:
train_csv.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24236806 entries, 0 to 24236805
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   Asset_ID   int64  
 2   Count      float64
 3   Open       float64
 4   High       float64
 5   Low        float64
 6   Close      float64
 7   Volume     float64
 8   VWAP       float64
 9   Target     float64
dtypes: float64(8), int64(2)
memory usage: 1.8 GB


# Memory Size Reduction

In [14]:
for column in train_csv:
    print(column)
    if train_csv[column].dtype == 'float64':
        train_csv[column]=pd.to_numeric(train_csv[column], downcast='float')
    if train_csv[column].dtype == 'int64':
        train_csv[column]=pd.to_numeric(train_csv[column], downcast='integer')

timestamp
Asset_ID
Count
Open
High
Low
Close
Volume
VWAP
Target


In [15]:
train_csv.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24236806 entries, 0 to 24236805
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int32  
 1   Asset_ID   int8   
 2   Count      float32
 3   Open       float32
 4   High       float32
 5   Low        float32
 6   Close      float32
 7   Volume     float32
 8   VWAP       float32
 9   Target     float32
dtypes: float32(8), int32(1), int8(1)
memory usage: 855.2 MB


# Dataset description

In [16]:
dtf_description = train_csv.describe()
dtf_description

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
count,2.423681e+07,2.423681e+07,2.423681e+07,2.423681e+07,2.423681e+07,2.423681e+07,2.423681e+07,2.423681e+07,2.423680e+07,2.348647e+07
mean,1.577120e+09,6.292544e+00,2.864594e+02,1.432640e+03,1.436351e+03,1.429568e+03,1.432642e+03,2.868529e+05,NaN,7.121750e-06
std,3.323350e+07,4.091861e+00,8.673981e+02,6.029605e+03,6.039482e+03,6.020261e+03,6.029611e+03,2.433935e+06,NaN,5.679042e-03
min,1.514765e+09,0.000000e+00,1.000000e+00,1.170400e-03,1.195000e-03,2.000000e-04,1.171400e-03,-3.662812e-01,-inf,-5.093510e-01
25%,1.549011e+09,3.000000e+00,1.900000e+01,2.676500e-01,2.681600e-01,2.669000e-01,2.676484e-01,1.410725e+02,2.676368e-01,-1.694354e-03
50%,1.578372e+09,6.000000e+00,6.400000e+01,1.428860e+01,1.431250e+01,1.426300e+01,1.428920e+01,1.295415e+03,1.428769e+01,-4.289844e-05
75%,1.606198e+09,9.000000e+00,2.210000e+02,2.288743e+02,2.293000e+02,2.284200e+02,2.288729e+02,2.729764e+04,2.288728e+02,1.601520e-03
max,1.632182e+09,1.300000e+01,1.650160e+05,6.480595e+04,6.490000e+04,6.467053e+04,6.480854e+04,7.597554e+08,inf,9.641699e-01


In [4]:
train_csv

pandas.core.frame.DataFrame

In [5]:
train_csv.iloc[1:10]

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
1,1514764860,0,5.0,8.530000,8.5300,8.5300,8.530000,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.194000,14013.8000,13666.1100,13850.176000,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.659600,7.6596,7.6567,7.657600,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.920000,25.9200,25.8740,25.877000,121.087310,25.891363,-0.008264
5,1514764860,6,173.0,738.302500,746.0000,732.5100,738.507500,335.987856,738.839291,-0.004809
6,1514764860,9,167.0,225.330000,227.7800,222.9800,225.206667,411.896642,225.197944,-0.009791
7,1514764860,11,7.0,329.090000,329.8800,329.0900,329.460000,6.635710,329.454118,NaN
8,1514764920,2,53.0,2374.553333,2400.9000,2354.2000,2372.286667,24.050259,2371.434498,-0.004079
9,1514764920,0,7.0,8.530000,8.5300,8.5145,8.514500,71.390000,8.520215,-0.015875


In [7]:
test_csv

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,group_num,row_id
0,1623542400,3,1201.0,1.478556,1.486030,1.478000,1.483681,6.547996e+05,1.481439,0,0
1,1623542400,2,1020.0,580.306667,583.890000,579.910000,582.276667,1.227988e+03,581.697038,0,1
2,1623542400,0,626.0,343.789500,345.108000,343.640000,344.598000,1.718833e+03,344.441729,0,2
3,1623542400,1,2888.0,35554.289632,35652.464650,35502.670000,35602.004286,1.638115e+02,35583.469303,0,3
4,1623542400,4,433.0,0.312167,0.312600,0.311920,0.312208,5.855774e+05,0.312154,0,4
5,1623542400,5,359.0,4.832550,4.845900,4.822900,4.837583,4.714355e+04,4.836607,0,5
6,1623542400,7,541.0,55.223080,55.494000,55.182000,55.344680,6.625202e+03,55.298816,0,6
7,1623542400,6,2186.0,2371.194286,2379.200000,2369.670000,2374.380714,1.214129e+03,2374.335307,0,7
8,1623542400,8,35.0,1.003150,1.019800,0.987300,1.003300,7.061928e+03,1.002936,0,8
9,1623542400,9,560.0,161.933429,162.480000,161.730000,162.214714,1.485009e+03,162.231310,0,9


In [8]:
samples_submission_csv

,group_num,row_id,Target
0,0,0,0
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,0
5,0,5,0
6,0,6,0
7,0,7,0
8,0,8,0
9,0,9,0


In [9]:
asset_csv = pd.read_csv("asset_details.csv")

In [10]:
asset_csv

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar
